<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preprocess-Scrapped-Data" data-toc-modified-id="Preprocess-Scrapped-Data-1">Preprocess Scrapped Data</a></span></li><li><span><a href="#PeopleOfInformatics.csv:" data-toc-modified-id="PeopleOfInformatics.csv:-2"><code>PeopleOfInformatics</code>.csv:</a></span><ul class="toc-item"><li><span><a href="#Check-for-duplicated-names" data-toc-modified-id="Check-for-duplicated-names-2.1">Check for duplicated <code>names</code></a></span></li><li><span><a href="#Check-the-institute-field" data-toc-modified-id="Check-the-institute-field-2.2">Check the <code>institute</code> field</a></span></li><li><span><a href="#Alias/shortnames" data-toc-modified-id="Alias/shortnames-2.3">Alias/shortnames</a></span></li></ul></li><li><span><a href="#publications.csv-:" data-toc-modified-id="publications.csv-:-3"><code>publications</code>.csv :</a></span><ul class="toc-item"><li><span><a href="#Assigning-unique-ID-to-external-academic" data-toc-modified-id="Assigning-unique-ID-to-external-academic-3.1">Assigning unique ID to external academic</a></span></li><li><span><a href="#Creating-index-for-collaboration:" data-toc-modified-id="Creating-index-for-collaboration:-3.2">Creating index for collaboration:</a></span></li></ul></li></ul></div>

---
# Preprocess Scrapped Data
In this notebook, we clean the data that was scrapped from Edinbugh Research Explorer for the School of Informatics. Preprocessing the data is critical especially for this dataset, as it was noted by many other researchers on collaboration networks that the data collected are usually fuzzy (the problem of alias being used inconsistently for many individuals will resurface later). 

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import os, csv

In [2]:
peopleOfInf_csv = '../data/peopleOfInformatics.csv'
publications_csv = '../data/publications.csv'

# `PeopleOfInformatics`.csv:
This file contains information about the individuals in the School of Informatics. Information scrapped includes the unique individual id, first and last name, indvidual's position in the school, and the institute they belong to.

In [3]:
# Import the csv:
pd_pplOfInf = \
pd.read_csv(
    filepath_or_buffer=peopleOfInf_csv,
    names=[
        'id', 'last_name', 'first_name', 'perseonal_url', 'position', 'parent',
        'institute'
    ],
    index_col=False)

# create a column for the full name (= last_name + full_name)
_join_names = lambda (x, y): " ".join([x, y])
pd_pplOfInf['full_name'] = list(
    map(_join_names, zip(pd_pplOfInf['last_name'], pd_pplOfInf['first_name'])))

print 'imported: ', len(pd_pplOfInf.index)
print 'number of indivudals =', len(pd.unique(pd_pplOfInf['full_name']))

imported:  296
number of indivudals = 292


## Check for duplicated `names`

In [4]:
dups_full_name = pd_pplOfInf.duplicated(subset='full_name', keep=False)
small = pd_pplOfInf[dups_full_name]
small

,id,last_name,first_name,perseonal_url,position,parent,institute,full_name
65,863ba3ac-4506-49fe-aa7f-5568cba5e388,elliott,desmond,http://www.research.ed.ac.uk/portal/en/persons...,visitor default visitor,school of informatics,UNKNOWN,elliott desmond
66,9333c02f-770b-4b9c-b3d2-ff1322fc99aa,elliott,desmond,http://www.research.ed.ac.uk/portal/en/persons...,research associate,school of informatics,institute of language cognition and computation,elliott desmond
96,003ec9bb-18aa-4e6e-95e9-359f0968262a,gray,gavin,http://www.research.ed.ac.uk/portal/en/persons...,research assistant,school of informatics,institute for adaptive and neural computation,gray gavin
97,11447e11-aff5-438e-8fb6-264da9fb5483,gray,gavin,http://www.research.ed.ac.uk/portal/en/persons...,unknown,neuroinformatics dtc,UNKNOWN,gray gavin
170,265079a1-b5d5-4ff1-bf46-d0ad26e0d424,maneth,sebastian,http://www.research.ed.ac.uk/portal/en/persons...,unknown,laboratory for foundations of computer science,UNKNOWN,maneth sebastian
171,ef703785-d96b-49af-9b95-df74fa8fc4a2,maneth,sebastian,http://www.research.ed.ac.uk/portal/en/persons...,visitor official visitor,school of informatics,UNKNOWN,maneth sebastian
248,0ed800f5-a3a0-47d7-a8b3-f97a4f2b6931,steuwer,michel,http://www.research.ed.ac.uk/portal/en/persons...,unknown,institute for computing systems architecture,UNKNOWN,steuwer michel
249,5ddb75bd-0bf1-4792-ba2f-52821719fc3b,steuwer,michel,http://www.research.ed.ac.uk/portal/en/persons...,visitor official visitor,school of informatics,UNKNOWN,steuwer michel


Manually checking the data gives us:

1) `elliott,desmond`
    - (65) does not have any research output
    - (66) is the main page with multiple research output

3) `gray gavin`
    - (96) is the main page with multiple research output
    - (97) does not have any research output

3) `maneth,sebastian`
    - (170) is the main page with multiple research output
    - (171) does not have any research output

4) `steuwer,michel`
    - (248) is the main page with multiple output
    - (249) does not have any research output



In [5]:
# manually remove index 65, 97, 171, 249
pd_pplOfInf.drop([65, 97, 171, 249], axis=0, inplace=True)

In [6]:
# Confirm that the number of individuals tallies:
print 'size of df: ', len(pd_pplOfInf.index)
print 'number of indivudals =', len(pd.unique(pd_pplOfInf['full_name']))

size of df:  292
number of indivudals = 292


## Check the `institute` field
There are multiple institutes field where `school of informatics` was present; while most are labelled `UNKNOWN` by the scrapper as no such information is present.

In [7]:
print 'BEFORE PROCESSING'
# separate the individuals by community:
infcommunity = {}
gb = pd_pplOfInf.groupby('institute')
for k, group in gb:
    print '{}: {}'.format(k, len(group))

BEFORE PROCESSING
UNKNOWN: 80
centre for intelligent systems and their applications: 19
deanery of clinical sciences: 1
edinburgh neuroscience: 1
institute for adaptive and neural computation: 29
institute for computing systems architecture: 24
institute of language cognition and computation: 59
institute of perception action and behaviour: 19
laboratory for foundations of computer science: 57
school of informatics: 2
school of philosophy psychology and language sciences: 1


The distribution of the individuals in the institutes is not ideal; some of the institutes name may fall into the parent field. To avoid such instances, we copy the `parent`'s field into the `institute`'s field if the entry in the parent's field belongs to one of the institute names

In [8]:
# check that the dataset for institute names:
# if parent is not 'school of informatics', and institute is unknown, copy it over:
parent = pd_pplOfInf['parent']
institute = pd_pplOfInf['institute']
for i, (par, inst) in enumerate(zip(parent, institute)):
    # We want to remove institute=school of informatics:
    if inst == 'UNKNOWN' or inst == 'school of informatics':
        if par == 'school of informatics':
            inst = 'UNKNOWN'
        else:
            inst = par
    #     pd_pplOfInf.institute[i] = inst
    pd_pplOfInf.loc[i, 'institute'] = inst

In [9]:
print 'AFTER PROCESSING'
# separate the individuals by community:
infcommunity = {}
gb = pd_pplOfInf.groupby('institute')
for k, group in gb:
    print '{}: {}'.format(k, len(group))

AFTER PROCESSING
UNKNOWN: 50
centre for intelligent systems and their applications: 23
deanery of clinical sciences: 1
edinburgh neuroscience: 1
institute for adaptive and neural computation: 30
institute for computing systems architecture: 28
institute of language cognition and computation: 63
institute of perception action and behaviour: 20
laboratory for foundations of computer science: 63
neuroinformatics dtc: 16
school of philosophy psychology and language sciences: 1


In [10]:
# Creat a mapping for the institutes to classes:
institutes = dict()
k = 0
for i in pd.unique(pd_pplOfInf['institute']):
    institutes[i] = k
    k += 1
print 'number of institutes found: {}'.format(len(institutes))

number of institutes found: 11


In [14]:
# Save the institutes info:
import cPickle as pkl
pkl.dump(institutes, open('../data/institutes.pkl', 'w'), pkl.HIGHEST_PROTOCOL)

In [11]:
# Set the class to each of the institute instance:
pd_pplOfInf['institute_class'] = pd_pplOfInf.apply(
    lambda row: institutes[row['institute']], axis=1)

In [12]:
pd_pplOfInf[pd_pplOfInf.institute=='school of informatics']

,id,last_name,first_name,perseonal_url,position,parent,institute,full_name,institute_class


In [13]:
# separate the individuals by community:
infcommunity = {}
gb = pd_pplOfInf.groupby('institute_class')
for k, group in gb:
    infcommunity[k] = group
    className = [name for (name, _k) in institutes.items() if _k == k][0]
    print 'class {}-{}: {}'.format(k, className, len(group))

class 0-UNKNOWN: 50
class 1-centre for intelligent systems and their applications: 23
class 2-institute of language cognition and computation: 63
class 3-laboratory for foundations of computer science: 63
class 4-institute for adaptive and neural computation: 30
class 5-institute for computing systems architecture: 28
class 6-neuroinformatics dtc: 16
class 7-institute of perception action and behaviour: 20
class 8-deanery of clinical sciences: 1
class 9-edinburgh neuroscience: 1
class 10-school of philosophy psychology and language sciences: 1


From the Edinburgh Research Explorer, we know that [school of informatics](http://www.research.ed.ac.uk/portal/en/organisations/school-of-informatics%28d9a3581f-93a4-4d74-bf29-14c86a1da9f4%29.html) have the following institutes:
- Centre for Intelligent Systems and their Applications (1)
- Institute of Language, Cogn)tion and Computation (2)
- Laboratory for Foundations of Computer Science (3)
- Institute for Adaptive and Neural Computation (4)
- Institute for Computing Systems Architecture (5)
- Neuroinformatics DTC (6)
- Institute of Perception, Action and Behaviour (7)

with the numbers in the brackets corresponding to the mapping in the institutes dictionary. This will be useful later for our visualisation and creation of subgraphs

## Alias/shortnames

In publications, author's name are not spelled in full when cited. Rather, they are represented in contracted forms, such as `Calautti, M., Libkin, L. & Pieris, A` instead of `Marco Calautti, Leonid Libkin, & Andreas Pieris` respectively. This could be a problem down stream as an individual's name (in publication) may presented in full for some, and in shortname for others. This make mapping the a certain publication to individuals a challenge.

To mitigate this, additional information (scrapped from Edinburgh research explorer) to assign shortnames (aliases) to an individual will be useful. These shortnames are assign to an individual in the school if the name is linked to their personal page on the research explorer

In [14]:
# assign the alias to each individuals, if exists:
poinf_alias = '../data/peopleOfInformatics_ALIAS.csv'
pd_poinf_alias = pd.read_csv(poinf_alias, names=['id', 'alias'])

In [15]:
# Merge the two pandas dataset to include the alias
lookup_poinf = pd_pplOfInf.set_index('id').join(pd_poinf_alias.set_index('id'))

In [16]:
lookup_poinf.iloc[[21,25,100,200]]

,last_name,first_name,perseonal_url,position,parent,institute,full_name,institute_class,alias
id,,,,,,,,,
102286ee-5f21-4aed-abfd-e4ea1a615223,oberlander,jon,http://www.research.ed.ac.uk/portal/en/persons...,professor,school of informatics,institute of language cognition and computation,oberlander jon,2,"oberlander, j."
12ea09ae-bba3-4ca6-b4e1-98e678043f8c,winn,john,http://www.research.ed.ac.uk/portal/en/persons...,visitor official visitor,school of informatics,institute of perception action and behaviour,winn john,7,NaN
55edc24c-73ac-4c72-9508-958445f08e5c,van rossum,mark,http://www.research.ed.ac.uk/portal/en/persons...,personal chair of computational neuroscience,school of informatics,institute of perception action and behaviour,van rossum mark,7,"rossum, m. v.|van rossum, m. c.|van rossum, m...."
b605a8d1-7d65-4380-8cb4-21004fad3dfc,leoni,david,http://www.research.ed.ac.uk/portal/en/persons...,unknown,centre for intelligent systems and their appli...,UNKNOWN,leoni david,0,"leoni, d."


In [17]:
lookup_poinf[lookup_poinf.last_name=='o\'boyle']

,last_name,first_name,perseonal_url,position,parent,institute,full_name,institute_class,alias
id,,,,,,,,,
1419562d-17ae-4ef2-9014-ca629eed6adb,o'boyle,michael,http://www.research.ed.ac.uk/portal/en/persons...,personal chair in computer science,school of informatics,institute of perception action and behaviour,o'boyle michael,7,"o'boyle, m. f. p.|o'boyle, m.|o'boyle, m. f. p..."


In [18]:
lookup_poinf[lookup_poinf.last_name=='perera']

,last_name,first_name,perseonal_url,position,parent,institute,full_name,institute_class,alias
id,,,,,,,,,
e76a7c40-a177-4eb5-9ac3-12ab104895f8,perera,roland,http://www.research.ed.ac.uk/portal/en/persons...,research associate,school of informatics,institute for computing systems architecture,perera roland,5,"perera, r."


In [23]:
# Write the lookup_poinf to the index:
lookup_poinf.to_csv(path_or_buf='../data/lookup_poinf.csv', index=True, index_label='id', header=True)

In [24]:
lookup_poinf.to_pickle(path='../data/lookup_poinf.pkl')

---
# `publications`.csv :
There are two csv files related to information from publications:
1. `publications.csv`
2. `pubpage_aliases.csv`

`publcations.csv` contains data scrapped from the each publication under the school of informatics. The fields are:`pub_id`,`date`,`year`,`title`,`authors`,`pub_url`,`doi_url`,`pdf_url`,`abstract`, and `publications`. For our collaboration network, we are only interested in the `pub_id`, `year` (for temporal study), and `authors`.

`pubpage_aliases.csv` contains mapping of `shortnames` of authors for each publication. As mentioned in the [previous section](#Alias/shortnames).

In [19]:
pd_publications = pd.read_csv(publications_csv) 
pubpage_alias = pd.read_csv('../data/pubpage_aliases.csv', names=['pub_id', 'names', 'shortnames'])
# merge the dataframes:
lookup_pub = pd_publications.set_index('pub_id').join(pubpage_alias.set_index('pub_id'))

In [21]:
lookup_pub.drop(
    labels=[
        'title', 'date', 'pub_url', 'pdf_url', 'doi_url', 'abstract',
        'publications', 'names'
    ],
    axis=1,
    inplace=True)

In [22]:
lookup_pub.head(10)

,year,authors,shortnames
pub_id,,,
400818dc-63af-4a26-80c5-906f98e1f8ab,1989,d.p. o'brien|r. galvao|m. keilhacker|e. lazzar...,"brien, d. p.|galvao, r.|keilhacker, m.|lazzaro..."
18b1a861-afef-4fff-bc80-d02e05be18c4,2013,paolo guagliardo|piotr wieczorek,"guagliardo, p.|wieczorek, p."
309fdbfc-227b-4588-9264-f0f4e3cadfcb,1994,p. buneman|l. libkin|d. suciu|v. tannen|l. wong,"buneman, p.|libkin, l.|suciu, d.|tannen, v. |w..."
d5814bab-5fc2-4c31-92b7-543c7ce75cb4,2012,p.l. de leon|m. pucher|j. yamagishi|i. hernaez...,"leon, p. l.|pucher, m.|yamagishi, j.|hernaez, ..."
880944d3-26db-4003-9186-130bf3202941,2014,navid nikaein|mahesh k. marina|saravana manick...,"nikaein, n.|marina, m. k.|manickam, s.|dawson,..."
57881982-3f10-4035-9dfb-9e76dcfbc00d,2007,nobuyuki bannai|robert b. fisher|alexander aga...,"bannai, n.|fisher, r. b.|agathos, a."
2fdf828d-62a7-48f0-bd3e-403e20735174,2010,moa johansson|lucas dixon|alan bundy,"johansson, m.|dixon, l. |bundy, a."
46253c7e-fc72-49a0-b7b2-c375d71969ee,2013,r. kumar|a. martinez|a. gonzalez,"kumar, r.|martinez, a. |gonzalez, a."
6b3f3c93-53e1-42b2-be11-66a7cede3a57,2016,roly perera|julien lange|simon j. gay,"perera, r.|lange, j. |gay, s. j."


In the previous section on alias for People of informatics, we attempt to unravel the aliases for each individuals. Our dataset, however, have many individuals who may not be from the school of informatics (external academics), but have collaborated with individuals from the school of informatics.

Our task for the external academics are:
1. Assign an unique ID for them
2. Unravel through the publication and assign ids for the publications

## Assigning unique ID to external academic

In [23]:
import re
pairs = set()
for (names, shortnames) in zip(lookup_pub.authors, lookup_pub.shortnames):
    _names = str(names).split('|')
    _shortnames = str(shortnames).split('|')
    _shortnames = map(lambda x: x.strip(), _shortnames)

    if len(_names) == len(_shortnames):
        _pairs = zip(_names, _shortnames)
        for p in _pairs:
            # simplistic search and match:
            _lastname1 = p[0].split(" ")[-1]
            _lastname2 = p[1].split(" ")[0]
            _lastname2 = re.findall(r'[a-z]*', _lastname2)[0]
            if _lastname1[-len(_lastname2):] == _lastname2:
                pairs.add(p)

allNames = [p[0] for p in list(pairs)]
allShortNames = [p[1] for p in list(pairs)]

In [24]:
print 'Total number of individuals in the collaboration network: ', str(len(allNames))

Total number of individuals in the collaboration network:  9977


In [25]:
# Combine all the alias for individuals in the school of informatics into a list
poinf_aliases = []
for aliases in lookup_poinf.alias:
    if type(aliases) is str:
        poinf_aliases.extend(aliases.split("|"))

In [26]:
external_aliases = set(allShortNames).difference(set(poinf_aliases))

In [27]:
print 'Number of external individuals in collaboration network: ', str(len(external_aliases))

Number of external individuals in collaboration network:  7679


In [33]:
# Give an unique id to each alias:
from itertools import count
from collections import defaultdict
lookup_ext_academic = defaultdict(count().next)

In [34]:
# Create a dictionary for fast lookup of individuals who are not in school of informatics
for i in list(external_aliases):
    lookup_ext_academic[i]

In [36]:
# create a dictionary for fast lookup of individuals in school of informatics
lookup_poinf_alias = {}
for pers_id, aliases in zip(lookup_poinf.index, lookup_poinf.alias):
    if type(aliases) is str:
        _aliases = aliases.split("|")
        for alias in _aliases:
            lookup_poinf_alias[str(alias).strip()] = pers_id

## Creating index for collaboration:
Now that we have an assigned ID for external individuals and individuals who are in school of informatics, we can use these IDs to represent these individuals for each collaboration.

In [37]:
def collabIds(_shortnames):
    # collaborators id
    collab_id = []
    try:
        for i in _shortnames.split("|"):
            i = i.strip()
            if i in lookup_poinf_alias:
                collab_id.append(lookup_poinf_alias[i])
            elif i in lookup_ext_academic:
                collab_id.append(lookup_ext_academic[i])
            else:
                print i
    except:
        return []
    return collab_id


lookup_pub['collab_id'] = lookup_pub.apply(
    lambda row: collabIds(row.shortnames), axis=1)

kifer, m.
sharman, j. l.
esposito, a.
bonin, f.
chetouani, m.
cohn, j. f.
fuhrmann, f.
gilmartin, e.
hammal, z.
heylen, d.
kaiser, r.
koutsombogera, m.
potamianos, a.
salah, a. a.
ito, k.
shinomiya, k.
ito, m.
boyan, g.
hartenstein, v.
harzsch, s.
heisenberg, m.
homberg, u.
jenett, a.
keshishian, h.
restifo, l. l.
roessler, w.
simpson, j. h.
strauss, r.
vosshall, l. b.
lowe, n.
rees, j. s.
roote, j.
ryder, e.
armean, i. m.
johnson, g.
drummond, e.
spriggs, h.
drummond, j.
magbanua, j. p.
naylor, h.
sanson, b.
bastock, r.
huelsmann, s.
trovisco, v.
landgraf, m.
cooper, h.
hansen, c.
phillips, r. g.
consortium, lilley
russell, s.
johnston, d.
antonioli, n.
coletta, s.
grossi, s.
virardi, e.
amari, s
beltrame, f.
bjaalie, j. g.
dalkara, t.
egan, g. f.
gonzalez, c.
grillner, s.
herz, a.
hoffmann, k
jaaskelainen, i.
koslow, s. h.
lee, s
matthiessen, l.
miller, p. l.
silva, f. m. d.
novak, m.
ravindranath, v.
ritz, r.
ruotsalainen, u.
sebestra, v.
subramaniam, s.
toga, a. w.
usui, s.
pelt, j

In [38]:
lookup_pub.head(10)

,year,authors,shortnames,collab_id
pub_id,,,,
400818dc-63af-4a26-80c5-906f98e1f8ab,1989,d.p. o'brien|r. galvao|m. keilhacker|e. lazzar...,"brien, d. p.|galvao, r.|keilhacker, m.|lazzaro...","[324, 1942, 560, 5635, 5099, a78a09b5-138e-407..."
18b1a861-afef-4fff-bc80-d02e05be18c4,2013,paolo guagliardo|piotr wieczorek,"guagliardo, p.|wieczorek, p.","[653c9723-b374-4ad3-9cef-0f7e7f45d812, 4953]"
309fdbfc-227b-4588-9264-f0f4e3cadfcb,1994,p. buneman|l. libkin|d. suciu|v. tannen|l. wong,"buneman, p.|libkin, l.|suciu, d.|tannen, v. |w...","[90396347-473b-4fba-8b1d-a1d25ea18b6b, a22b33f..."
d5814bab-5fc2-4c31-92b7-543c7ce75cb4,2012,p.l. de leon|m. pucher|j. yamagishi|i. hernaez...,"leon, p. l.|pucher, m.|yamagishi, j.|hernaez, ...","[1180, 5318, 4d11a99a-0d04-42f4-8089-d433a57c2..."
880944d3-26db-4003-9186-130bf3202941,2014,navid nikaein|mahesh k. marina|saravana manick...,"nikaein, n.|marina, m. k.|manickam, s.|dawson,...","[3859, 06760916-edca-488e-93da-baff6204a453, 2..."
57881982-3f10-4035-9dfb-9e76dcfbc00d,2007,nobuyuki bannai|robert b. fisher|alexander aga...,"bannai, n.|fisher, r. b.|agathos, a.","[4666, 670f7376-cd3d-4eb5-8a57-736d6c988f42, 6..."
2fdf828d-62a7-48f0-bd3e-403e20735174,2010,moa johansson|lucas dixon|alan bundy,"johansson, m.|dixon, l. |bundy, a.","[5545, 414, bfb547ed-a4df-4a6c-b1be-8ecd9b13419f]"
46253c7e-fc72-49a0-b7b2-c375d71969ee,2013,r. kumar|a. martinez|a. gonzalez,"kumar, r.|martinez, a. |gonzalez, a.","[eb085c28-d4a0-4d51-83e8-881a148e7fff, 1453, 3..."
6b3f3c93-53e1-42b2-be11-66a7cede3a57,2016,roly perera|julien lange|simon j. gay,"perera, r.|lange, j. |gay, s. j.","[e76a7c40-a177-4eb5-9ac3-12ab104895f8, 6456, 2..."


In [39]:
# Create egelist
from itertools import product


def generateEdges(ids):
    products = product(ids, ids)
    differentPairs = []
    for (i, v) in products:
        if i != v and (v, i) not in differentPairs:
            differentPairs.append((i, v))
    return differentPairs

In [40]:
lookup_pub['edges'] = lookup_pub.apply(lambda row: generateEdges(row.collab_id), axis=1)

In [41]:
lookup_pub.iloc[0].authors

"d.p. o'brien|r. galvao|m. keilhacker|e. lazzaro|m.l. watkins|c.m. bishop"

In [42]:
lookup_pub.iloc[0].edges

[(324, 1942),
 (324, 560),
 (324, 5635),
 (324, 5099),
 (324, 'a78a09b5-138e-407e-8c09-34414853eb20'),
 (1942, 560),
 (1942, 5635),
 (1942, 5099),
 (1942, 'a78a09b5-138e-407e-8c09-34414853eb20'),
 (560, 5635),
 (560, 5099),
 (560, 'a78a09b5-138e-407e-8c09-34414853eb20'),
 (5635, 5099),
 (5635, 'a78a09b5-138e-407e-8c09-34414853eb20'),
 (5099, 'a78a09b5-138e-407e-8c09-34414853eb20')]

In [43]:
# Write the dataset to csv file for portability:
lookup_pub.to_csv(path_or_buf='../data/lookup_pub.csv',header=True,index=True,index_label='pub_id')

In [44]:
lookup_pub.to_pickle(path='../data/lookup_pub.pkl')